## Preprocessing

In [37]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [38]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
new_application_df = application_df.copy()
new_application_df.drop(columns=["EIN", "NAME"], axis=1, inplace=True)
new_application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [39]:
# Determine the number of unique values in each column.
application_cat = new_application_df.dtypes[(new_application_df.dtypes=="object")|
 (new_application_df.dtypes=="int64")].index.tolist()
new_application_df[application_cat].nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [40]:
# Look at APPLICATION_TYPE value counts for binning
new_application_df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [41]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 500
value_counts = new_application_df["APPLICATION_TYPE"].value_counts()
application_types_to_replace = list(value_counts[value_counts < cutoff_value].index)

# Replace in dataframe
for app in application_types_to_replace:
    new_application_df['APPLICATION_TYPE'] = new_application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
new_application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [42]:
# Look at CLASSIFICATION value counts for binning
new_application_df["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [43]:
# You may find it helpful to look at CLASSIFICATION value counts >1
new_application_df["CLASSIFICATION"].value_counts()[new_application_df["CLASSIFICATION"].value_counts()>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [44]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_value = 1000
value_counts = new_application_df["CLASSIFICATION"].value_counts()
classifications_to_replace = list(value_counts[value_counts < cutoff_value].index)

# Replace in dataframe
for cls in classifications_to_replace:
    new_application_df['CLASSIFICATION'] = new_application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
new_application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [45]:
# Convert categorical data to numeric with `pd.get_dummies`
new_application_df = pd.get_dummies(new_application_df)

In [46]:
# Split our preprocessed data into our features and target arrays
y=new_application_df["IS_SUCCESSFUL"].values
X=new_application_df.drop(["IS_SUCCESSFUL"],1).values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

<ipython-input-46-b75fdf5bae4c>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X=new_application_df.drop(["IS_SUCCESSFUL"],1).values


In [47]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

##Optimize Model: Option 1 - Add Hidden Layer and Increase Neurons

In [48]:
# OPTION1: Add hidden layers and increase neurons
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 50
hidden_nodes_layer3 = 30

nn1 = tf.keras.models.Sequential()

# First hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn1.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 80)                3520      
                                                                 
 dense_21 (Dense)            (None, 50)                4050      
                                                                 
 dense_22 (Dense)            (None, 30)                1530      
                                                                 
 dense_23 (Dense)            (None, 1)                 31        
                                                                 
Total params: 9,131
Trainable params: 9,131
Non-trainable params: 0
_________________________________________________________________


In [49]:
# Compile the model
nn1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn1.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 7s 5ms/step - loss: 0.5678 - accuracy: 0.7247
Epoch 2/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5561 - accuracy: 0.7300
Epoch 3/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5528 - accuracy: 0.7314
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5509 - accuracy: 0.7322
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5506 - accuracy: 0.7327
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5497 - accuracy: 0.7335
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5482 - accuracy: 0.7345
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5476 - accuracy: 0.7345
Epoch 9/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5473 - accuracy: 0.7353
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5461 - accura

In [50]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5609 - accuracy: 0.7252 - 490ms/epoch - 2ms/step
Loss: 0.5608916282653809, Accuracy: 0.7252478003501892


## Optimize Model: Option 2 - Change Activation and Add More Neurons

In [51]:
# OPTION2: Change Activation
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 80
hidden_nodes_layer3 = 80

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh"))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Third hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 80)                3520      
                                                                 
 dense_25 (Dense)            (None, 80)                6480      
                                                                 
 dense_26 (Dense)            (None, 80)                6480      
                                                                 
 dense_27 (Dense)            (None, 1)                 81        
                                                                 
Total params: 16,561
Trainable params: 16,561
Non-trainable params: 0
_________________________________________________________________


In [52]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn2.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5709 - accuracy: 0.7220
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5586 - accuracy: 0.7306
Epoch 3/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5546 - accuracy: 0.7297
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5528 - accuracy: 0.7304
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5512 - accuracy: 0.7321
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5500 - accuracy: 0.7334
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5480 - accuracy: 0.7332
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5476 - accuracy: 0.7322
Epoch 9/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5473 - accuracy: 0.7336
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5458 - accura

In [53]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5575 - accuracy: 0.7254 - 487ms/epoch - 2ms/step
Loss: 0.5574706196784973, Accuracy: 0.7253644466400146


## Optimize Model: Option 3 - Drop Variables

In [54]:
#OPTION3: Drop Variables
import numpy as np

# Calculate the importance of each variable
weights = nn2.get_weights()[0]
importance = np.abs(weights).sum(axis=1)

# Print importance of variables in descendig order
columns = new_application_df.columns[:-1]
for i in range(len(columns)):
  print(columns[i], importance[i])

STATUS 16.076733
ASK_AMT 47.979294
IS_SUCCESSFUL 26.740135
APPLICATION_TYPE_Other 26.826899
APPLICATION_TYPE_T10 25.642084
APPLICATION_TYPE_T19 20.213825
APPLICATION_TYPE_T3 24.028612
APPLICATION_TYPE_T4 28.655811
APPLICATION_TYPE_T5 30.39479
APPLICATION_TYPE_T6 25.243908
APPLICATION_TYPE_T7 23.877201
APPLICATION_TYPE_T8 27.568296
AFFILIATION_CompanySponsored 20.56048
AFFILIATION_Family/Parent 28.751759
AFFILIATION_Independent 20.876783
AFFILIATION_National 17.73502
AFFILIATION_Other 15.238314
AFFILIATION_Regional 28.604244
CLASSIFICATION_C1000 30.535063
CLASSIFICATION_C1200 35.826847
CLASSIFICATION_C2000 27.442072
CLASSIFICATION_C2100 27.515457
CLASSIFICATION_C3000 33.51233
CLASSIFICATION_Other 24.065336
USE_CASE_CommunityServ 18.816704
USE_CASE_Heathcare 15.518409
USE_CASE_Other 22.318047
USE_CASE_Preservation 24.809021
USE_CASE_ProductDev 30.88905
ORGANIZATION_Association 23.913948
ORGANIZATION_Co-operative 18.022348
ORGANIZATION_Corporation 30.65842
ORGANIZATION_Trust 21.99218
INCO

In [65]:
# Drop low value: STATUS, SPECIAL_CONSIDERATIONS
dropped_application_df = new_application_df.copy()
dropped_application_df.drop(columns=["STATUS", "SPECIAL_CONSIDERATIONS_N", "SPECIAL_CONSIDERATIONS_Y"], axis=1, inplace=True)
dropped_application_df.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,108590,1,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,5000,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3,6692,1,0,0,0,1,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,142590,1,0,0,0,1,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [66]:
# Split our preprocessed data into our features and target arrays
y1=dropped_application_df["IS_SUCCESSFUL"].values
X1=dropped_application_df.drop(["IS_SUCCESSFUL"],1).values
# Split the preprocessed data into a training and testing dataset
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, random_state=78)

<ipython-input-66-12035f51519f>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X1=dropped_application_df.drop(["IS_SUCCESSFUL"],1).values


In [67]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X1_scaler = scaler.fit(X1_train)

# Scale the data
X1_train_scaled = X1_scaler.transform(X1_train)
X1_test_scaled = X1_scaler.transform(X1_test)

In [79]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X1_train_scaled[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 80
hidden_nodes_layer3 = 80

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_44 (Dense)            (None, 80)                3280      
                                                                 
 dense_45 (Dense)            (None, 80)                6480      
                                                                 
 dense_46 (Dense)            (None, 80)                6480      
                                                                 
 dense_47 (Dense)            (None, 1)                 81        
                                                                 
Total params: 16,321
Trainable params: 16,321
Non-trainable params: 0
_________________________________________________________________


In [80]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn3.fit(X1_train_scaled,y1_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 4s 3ms/step - loss: 0.5699 - accuracy: 0.7215
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5537 - accuracy: 0.7303
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5519 - accuracy: 0.7338
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5500 - accuracy: 0.7331
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5491 - accuracy: 0.7340
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5481 - accuracy: 0.7340
Epoch 7/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5472 - accuracy: 0.7343
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5464 - accuracy: 0.7358
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5460 - accuracy: 0.7355
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5453 - accura

In [81]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X1_test_scaled,y1_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5692 - accuracy: 0.7266 - 509ms/epoch - 2ms/step
Loss: 0.5691957473754883, Accuracy: 0.7266472578048706


In [83]:
# Export our model to HDF5 file
nn1.save('AlphabetSoupCharity_Optimization1.h5')
nn2.save('AlphabetSoupCharity_Optimization2.h5')
nn3.save('AlphabetSoupCharity_Optimization3.h5')